# Import Libraries

This has do be executed once in the beginning. The libaries loaded include functions for numerical operations, scientific plotting and image manipulation.

In [ ]:
# NumPy – The fundamental package for scientific computing with Python
# https://numpy.org/
import numpy as np

# Matplotlib – Visualization with Python
# https://matplotlib.org/
#for interactive plots
%matplotlib inline 
#When exporting to pdf rplace the "notebook" with "inline" and replot every plot.
#This replaces the interactive plots with regular ones that can be converted with LaTeX.
import matplotlib.pyplot as plt

# Python Imaging Library (PIL)
# consider using the fork pillow
# https://pillow.readthedocs.io/en/stable/index.html
from PIL import Image

# SciPy – Python-based ecosystem of open-source software for mathematics, science, and engineering
# https://www.scipy.org/
from scipy.ndimage.interpolation import rotate

# Load the Data

Please specify the **entire file path and the file name**. Every time you want to analyze a new image, you have to load it here and then execute every step.

In [ ]:
file_folder = r"\\" #directory to the folder where the images recorded by the camera are stored
prefix = '' #please change the prefix name according to the run image name given by the camera software

save_folder = r'\\' #directory to the folder where you want to save the images. Make sure it exists!
prefix_save = '' #prefix that is added to the saved images. Change it as you like it.

numbers = [0,99] #first and last number of the image you want to load

for i in range(numbers[0], numbers[1] + 1):
    tmp = Image.open(file_folder + prefix + '%04d.tif'%i) 
    tmp = np.asarray(tmp.getdata(),dtype=np.float64).reshape((tmp.size[1],tmp.size[0]))
    if i == 0:
        holo_exp = tmp
    else:
        holo_exp += tmp

#the FFT needs a quadratic image as input
x,y = holo_exp.shape
if x<y:
    holo_exp = holo_exp[:, np.int((y-x)/2):-np.int((y-x)/2)]
elif x>y:
    holo_exp = holo_exp[np.int((x-y)/2):-np.int((x-y)/2),:]

## Plot the image

Plot the summed hologram.

You can adjust the scale of the image by specifying the mininal (*vmin*) and maximal (*max*) displayed value. You can do this with choosing the percentiles inthe first cell but you can also type in any number you like.

In [ ]:
MinV, MaxV = np.percentile(holo_exp,(2.5, 97.5))

fig, ax = plt.subplots()
ax = plt.imshow(holo_exp, cmap='gray', vmin = MinV, vmax = MaxV)
plt.colorbar()
plt.title('Hologram')

fig.tight_layout()

# Reconstruct

Now the artificial hologram is reconstructed. This is the only step that you will execute with the experimental data.

In [ ]:
recon_exp = np.abs(np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(holo_exp))))

## Plot the image

As before...

In [ ]:
MinV, MaxV = np.percentile(recon_exp,(1,99))

fig, ax = plt.subplots()

ax = plt.imshow(recon_exp, cmap = 'gray', vmin = MinV, vmax = MaxV)
plt.colorbar()
plt.title('Reconstruction')

fig.tight_layout()

#save image and data
plt.savefig(save_folder + prefix_save + 'recon_exp.png')
np.save(save_folder + prefix_save + 'recon_exp', recon_exp) #saves as numpy array. can be opened in another notebook with np.load('filepath')

# Lineout

Do reasonable intensity lineouts to determine the resolution using the 90%-10% criterion.

You can determine the pixel positions by using the cursor over your image. You can also select an area of the image to enlarge it using the butten left of the floppy disc.

If your reconstruction is not horizontally aligned, you need to rotate your image with the first cell.

In [ ]:
#rotate the image if it is not horizontally aligned
rot = rotate(recon_exp, 0) #inut the image as numpy array and the angle in degree

fig, ax = plt.subplots()
ax.imshow(rot, cmap = 'gray')

In [ ]:
def lineout(image, startX, endX, startY, endY, file_prefix):
    lineout = np.sum(image[startY:endY, startX:endX], 0)
    
    #plot the lineout
    fig, ax = plt.subplots()
    ax = plt.plot(lineout)
    plt.title("Lineout for resultion")
    plt.xlabel("Pixel")
    plt.ylabel("Intensity [a.u.]")
    
    #save the lineout
    np.savetxt(save_folder + file_prefix + 'lineout.txt', lineout)
    return

In [ ]:
#if you rotated you image, please put in the rotated image instead of "image"
lineout(recon_exp, startX = , endX = , startY = , endY = , file_prefix= prefix_save + 'exp')